In [1]:
import pandas as pd
from pmdarima import auto_arima
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv("급속완속충전기_누적대수2024삭제.csv")

In [3]:
data

,Unnamed: 0,자치구,년도,급속,완속
0,0,강남구,2013,0,2
1,1,강남구,2017,34,93
2,2,강남구,2018,54,196
3,3,강남구,2019,70,290
4,4,강남구,2020,89,438
...,...,...,...,...,...
203,227,중랑구,2019,25,153
204,228,중랑구,2020,33,223
205,229,중랑구,2021,53,457
206,230,중랑구,2022,103,1315


In [6]:
# 결과를 저장할 데이터프레임 초기화
forecast_results = []

# 자치구 리스트 추출
districts = data['자치구'].unique()

# 변수 리스트 (예측하고자 하는 변수들)
variables = [
    "급속","완속"]

# 각 자치구별로 모델 생성 및 예측
for district in districts:
    district_data = data[data['자치구'] == district]
    
    for variable in variables:
        if variable not in district_data.columns:
            continue

        # 연도별 데이터 정리
        ts_data = district_data[['년도', variable]].set_index('년도')
        ts_data = ts_data.dropna()  # 결측값 제거

        # Auto ARIMA 모델 학습
        model = auto_arima(ts_data, seasonal=True, m=1, suppress_warnings=True, start_p=0, d=1, start_q=0, max_p=3, max_q=3, 
                            enforce_stationarity=False)

        # 예측값이 음수가 나오지 않도록 처리
        def ensure_positive_forecast(forecast):
            return [max(0, value) for value in forecast]

        
        # 예측 (2025, 2026년)
        future_years = [2024,2025, 2026]
        forecast = ensure_positive_forecast(model.predict(n_periods=len(future_years)))

        # 결과 저장
        for year, value in zip(future_years, forecast):
            forecast_results.append({
                '자치구': district,
                '년도': year,
                '변수': variable,
                '예측값': value
            })

# 결과를 데이터프레임으로 변환
forecast_df = pd.DataFrame(forecast_results)

C:\Users\dyj02\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\dyj02\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\dyj02\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\dyj02\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_predicti

In [7]:
forecast_df

,자치구,년도,변수,예측값
0,강남구,2024,급속,391.655615
1,강남구,2025,급속,452.871274
2,강남구,2026,급속,490.234978
3,강남구,2024,완속,5048.578665
4,강남구,2025,완속,6200.158329
...,...,...,...,...
145,중랑구,2025,급속,500.840482
146,중랑구,2026,급속,659.284176
147,중랑구,2024,완속,1895.500000
148,중랑구,2025,완속,2106.000000


In [8]:
forecast_df.to_csv("급속완속예측.csv", index=False)